# Explainable AI (XAI) Methods Implementation
# Using Iris Dataset

In [ ]:
!pip install xgboost shap lime scikit-learn pandas numpy matplotlib seaborn --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import lime
import lime.lime_tabular
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb

# Set random seed for reproducibility
np.random.seed(42)

# 2. Load the Iris Dataset

In [ ]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name='target')

# 3. Data Preprocessing

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 4. Model Training

In [ ]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

# 5. Model Evaluation

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# 6. Explainable AI Interpretations

## 6.1 SHAP (SHapley Additive exPlanations)

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test, feature_names=iris.feature_names)

## 6.2 LIME (Local Interpretable Model-Agnostic Explanations)

In [ ]:
explainer_lime = lime.lime_tabular.LimeTabularExplainer(
    X_train,
    feature_names=iris.feature_names,
    class_names=iris.target_names,
    discretize_continuous=True
)

idx = 5  # Pick a test instance
exp = explainer_lime.explain_instance(X_test[idx], model.predict_proba, num_features=4)
exp.show_in_notebook(show_table=True)

## 6.3 Permutation Feature Importance

In [ ]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)

feature_importance = pd.DataFrame({
    'Feature': iris.feature_names,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Permutation Feature Importance')
plt.show()

# 7. Conclusion

In this notebook, we've demonstrated how to apply three different Explainable AI (XAI) techniques to interpret a machine learning model:

**SHAP (SHapley Additive exPlanations):**
- Provided global feature importance through summary plots
- Visualized feature interactions and dependencies
- Showed the impact of features on individual predictions

**LIME (Local Interpretable Model-Agnostic Explanations):**
- Generated locally faithful explanations for individual predictions
- Showed which features were most important for specific predictions

**Permutation Feature Importance:**
- Measured the decrease in model performance when features are permuted
- Compared permutation importance with the model's built-in feature importance

These techniques give us different perspectives on model interpretability:
- SHAP provides a mathematically rigorous approach to understanding both global and local feature importance
- LIME focuses on local explanations that are easy to understand
- Permutation importance offers a model-agnostic way to measure feature importance

**Key findings:**
- The most important features according to all three methods were related to petal width, petal length, and sepal width
- SHAP and permutation importance showed similar rankings for top features
- The model performed well on the classification task with high accuracy

XAI techniques help build trust in machine learning models by making their decision-making process more transparent and understandable. This is especially important in domains where model decisions have significant consequences.